In [ ]:
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

In [ ]:
model = ChatOllama(model = "qwen3:1.7b")

In [ ]:
class WorkflowState(TypedDict):
    topic: str
    blog_outline: str
    final_blog: str

In [ ]:
def generateBlogOutline(state: WorkflowState) -> WorkflowState:
    topic = state['topic']
    outline_generation_prompt = f"Generate a detailed outiline on the following topic {topic} for a blog"
    detailed_outline = model.invoke(outline_generation_prompt).content
    state['blog_outline'] = detailed_outline
    return state
    
    

In [ ]:
def generateBlog(state: WorkflowState) -> WorkflowState:
    topic = state['topic']
    detailed_blog_outline = state['blog_outline']
    blog_generation_prompt = f"Generate a blog on the following topic {topic} and the provided outline {detailed_blog_outline} for it"
    blog = model.invoke(blog_generation_prompt).content
    state['final_blog'] = blog
    return state

In [ ]:
# Graph design
graph = StateGraph(WorkflowState)
graph.add_node('generate_blog_outline', generateBlogOutline)
graph.add_node('generate_blog', generateBlog)

graph.add_edge(START, 'generate_blog_outline')
graph.add_edge('generate_blog_outline', 'generate_blog')
graph.add_edge('generate_blog', END)

workflow = graph.compile()


In [ ]:
initial_state = {"topic": "Recent trends in technology"}
final_state = workflow.invoke(initial_state)
print(final_state)

In [ ]:
print(final_state['blog_outline'])

In [ ]:
print(final_state['final_blog'])

In [ ]:
from IPython.display import Image
Image(workflow.get_graph().draw_mermaid_png())